# Database programming: SQLite

SQLite is an embedded database engine (there is no separate server) that is included with Python.

In [1]:
import sqlite3
conn = sqlite3.connect('data/real-estate.db')

In [2]:
cursor = conn.execute('SELECT * FROM "transactions" LIMIT 10')

In [3]:
cursor.description

(('index', None, None, None, None, None, None),
 ('street', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('zip', None, None, None, None, None, None),
 ('state', None, None, None, None, None, None),
 ('beds', None, None, None, None, None, None),
 ('baths', None, None, None, None, None, None),
 ('sq__ft', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None),
 ('sale_date', None, None, None, None, None, None),
 ('price', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None))

In [4]:
for row in cursor:
    print(row)

(0, '3526 HIGH ST', 'SACRAMENTO', 95838, 'CA', 2, 1, 836, 'Residential', 'Wed May 21 00:00:00 EDT 2008', 59222, 38.631913, -121.43487900000001)
(1, '51 OMAHA CT', 'SACRAMENTO', 95823, 'CA', 3, 1, 1167, 'Residential', 'Wed May 21 00:00:00 EDT 2008', 68212, 38.478902000000005, -121.431028)
(2, '2796 BRANCH ST', 'SACRAMENTO', 95815, 'CA', 2, 1, 796, 'Residential', 'Wed May 21 00:00:00 EDT 2008', 68880, 38.618305, -121.44383899999998)
(3, '2805 JANETTE WAY', 'SACRAMENTO', 95815, 'CA', 2, 1, 852, 'Residential', 'Wed May 21 00:00:00 EDT 2008', 69307, 38.616834999999995, -121.439146)
(4, '6001 MCMAHON DR', 'SACRAMENTO', 95824, 'CA', 2, 1, 797, 'Residential', 'Wed May 21 00:00:00 EDT 2008', 81900, 38.51947, -121.435768)
(5, '5828 PEPPERMILL CT', 'SACRAMENTO', 95841, 'CA', 3, 1, 1122, 'Condo', 'Wed May 21 00:00:00 EDT 2008', 89921, 38.662595, -121.32781299999999)
(6, '6048 OGDEN NASH WAY', 'SACRAMENTO', 95842, 'CA', 3, 2, 1104, 'Residential', 'Wed May 21 00:00:00 EDT 2008', 90895, 38.681659, -1

In [5]:
for row in conn.execute('SELECT type, count(*) from transactions GROUP BY type'):
    print(row)

('Condo', 54)
('Multi-Family', 13)
('Residential', 917)
('Unkown', 1)


"Bind" parameters:

In [6]:
stmt = 'SELECT count(*), avg(price) FROM transactions WHERE type = ?'

In [7]:
for row in conn.execute(stmt, ('Condo',)):
    print(row)

(54, 150082.1851851852)


In [8]:
for row in conn.execute(stmt, ('Residential',)):
    print(row)

(917, 239186.1624863686)


Writing data

In [9]:
c = conn.cursor()
c.execute("""CREATE TABLE stocks(
    date text, 
    symbol text, 
    price real
)""")

In [10]:
data = [
    ("2014-01-02", "F", 12.089),
    ("2014-01-02", "TSLA", 150.1),
    ("2014-01-02", "IBM", 157.6001),
    ("2014-01-02", "AAPL", 72.7741),
    ("2014-01-03", "F", 12.1438),
    ("2014-01-03", "TSLA", 149.56),
    ("2014-01-03", "IBM", 158.543),
    ("2014-01-03", "AAPL", 71.1756),
    ("2014-01-06", "F", 12.1986),
    ("2014-01-06", "TSLA", 147.0),
    ("2014-01-06", "IBM", 157.9993),
    ("2014-01-06", "AAPL", 71.5637),
    ("2014-01-07", "F", 12.042),
    ("2014-01-07", "TSLA", 149.36),
    ("2014-01-07", "IBM", 161.1508),
    ("2014-01-07", "AAPL", 71.0516),
    ("2014-01-08", "F", 12.1673),
    ("2014-01-08", "TSLA", 151.28),
    ("2014-01-08", "IBM", 159.6728),
    ("2014-01-08", "AAPL", 71.5019),
]

In [11]:
c.executemany('INSERT INTO stocks VALUES(?, ?, ?)', data)

In [12]:
conn.commit()

In [13]:
with conn:
    conn.execute("DELETE FROM stocks WHERE symbol = 'TSLA'")

In [14]:
c.execute("DROP TABLE stocks")

# Database programming: MySQL

To use MySQL, we'll need to install the pymysql driver:

In [15]:
!pip install pymysql

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:
import pymysql
import getpass

host = 'classdb.c9zriyyu6mtc.us-east-1.rds.amazonaws.com'
master_conn = pymysql.connect(
    host=host,
    user='master',
    password=getpass.getpass('Master Password?'),
    db='class'
)

Master Password?········


OperationalError: (2003, "Can't connect to MySQL server on 'classdb.c9zriyyu6mtc.us-east-1.rds.amazonaws.com' (timed out)")

In [ ]:
with master_conn.cursor() as c:
    c.execute(f"CREATE USER 'newuser' IDENTIFIED BY 'password'")
    c.execute(f"GRANT ALL PRIVILEGES ON class . * TO 'newuser'")

In [ ]:
import pandas as pd
dat = pd.read_csv('data/states.csv')
dat.head()

In [ ]:
conn = pymysql.connect(
    host=host,
    user='newuser',
    password='password',
    db='class'
)

In [ ]:
with conn.cursor() as c:
    value = c.execute('SELECT type, count(*) from transactions GROUP BY type')
    for item in c:
        print(item)

In [ ]:
with conn.cursor() as c:
    c.execute("""CREATE TABLE stocks(
        date text, 
        symbol text, 
        price real
    )""")

In [ ]:
with conn.cursor() as c:
    c.executemany('INSERT INTO stocks VALUES(%s, %s, %s)', data)

In [ ]:
with conn.cursor() as c:
    c.execute("DELETE FROM stocks WHERE symbol = 'TSLA'")

In [ ]:
with conn.cursor() as c:
    c.execute("DROP TABLE stocks")

Open [DBAPI lab](./dbapi-lab.ipynb)

(after the lab)

In [ ]:
with master_conn.cursor() as c:
    c.execute(f"DROP USER 'newuser'")